# INSTALAR LIBRERIAS

In [1]:
# !pip install docx2pdf
#pip install ipywidgets

# IMPORTAR

In [1]:
from docx2pdf import convert
import os
import time

# CONSTRUIR LA RUTA DE LOS ARCHIVOS

In [2]:
def get_unidad_local():
    try:
        # ruta del archivo que contiene la información necesaria
        with open('../../../../unidad_local.txt', 'r') as file:
            unidad = file.readline().strip()
            return unidad
    except FileNotFoundError:
        print("El archivo 'unidad_local.txt' no se encuentra en la ruta especificada.")
    return None

def get_ruta_local():
    try:
        with open('../../../ruta_local.txt', 'r') as file:
            ruta = file.readline().strip()
            return ruta
    except FileNotFoundError:
        print("El archivo 'ruta_local.txt' no se encuentra en la ruta especificada.")
    return None

In [3]:
unidad = get_unidad_local()
ruta = get_ruta_local()
ruta_completa = os.path.join(unidad, ruta)
ruta_completa

'C:\\Documents\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON'

In [4]:
# colocar aqui la ruta de los archivos
path = ruta_completa + r'/informes'
contenido = os.listdir(path)
rutas =[]
i = 0
for name in contenido:
    rutas.append(path + '/' + name)
    i+=1

print('SE ENCONTRARON ' + str(i) + ' INFORMES')
print(len(rutas), 'INFORMES')
rutas

SE ENCONTRARON 10 INFORMES
10 INFORMES


['C:\\Documents\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/11860_CRS_22-05-2025_ROCA SERRANO ARMANDO_471.docx',
 'C:\\Documents\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/13067_CRS_29-05-2025_SAUCEDO CERRUTI SONIA_467.docx',
 'C:\\Documents\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/15195_CRS_21-05-2025_VEIZAGA ENRIQUEZ GUSTAVO_507.docx',
 'C:\\Documents\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/16213_CRS_23-05-2025_ZAPATA SERON SIMON_510.docx',
 'C:\\Documents\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/2893_CRS_30-05-2025_DAVALOS SILVA MARIO_511.docx',
 'C:\\Documents\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/3473_CRS_29-05-2025_ESCALERA FRANCO JORGE_466.docx',
 'C:\\Documents\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/41834_CRS_28-05-2025_CALVI PADILLA JUDITH_555.docx',
 'C:\\Documents\\OneDrive - Ingenio Azucarero Guabira 

# INICIA EL PROCESO DE CONVERCIÓN

In [5]:
rutas[1].replace('.docx','.pdf').replace('informes','pdf')

'C:\\Documents\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/pdf/13067_CRS_29-05-2025_SAUCEDO CERRUTI SONIA_467.pdf'

In [6]:
rutas[1].replace('.docx','.pdf').replace('informes','pdf')

'C:\\Documents\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/pdf/13067_CRS_29-05-2025_SAUCEDO CERRUTI SONIA_467.pdf'

In [11]:
i = 1
for filename in rutas:
    print('CONVIRTIENDO... INFORME ' + str(i) + ' ' + filename)
    # se realiza la convrsion pasando parametro de salida y entrada, 
    # para la salida se reemplaza la extencion y la ruta
    print(filename)
    print(filename.replace('.docx','.pdf').replace('informes','pdf'))
    pdf = filename.replace('.docx','.pdf').replace('informes','pdf').split('/')[-1]
    convert(filename, r'C:\Users\bismarksr\Downloads' + pdf)
    i+=1
    time.sleep(5)
print('END')

CONVIRTIENDO... INFORME 1 C:\Documents\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/11860_CRS_22-05-2025_ROCA SERRANO ARMANDO_471.docx
C:\Documents\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/11860_CRS_22-05-2025_ROCA SERRANO ARMANDO_471.docx
C:\Documents\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/pdf/11860_CRS_22-05-2025_ROCA SERRANO ARMANDO_471.pdf


  0%|          | 0/1 [00:00<?, ?it/s]

CONVIRTIENDO... INFORME 2 C:\Documents\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/13067_CRS_29-05-2025_SAUCEDO CERRUTI SONIA_467.docx
C:\Documents\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/13067_CRS_29-05-2025_SAUCEDO CERRUTI SONIA_467.docx
C:\Documents\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/pdf/13067_CRS_29-05-2025_SAUCEDO CERRUTI SONIA_467.pdf


  0%|          | 0/1 [00:00<?, ?it/s]

CONVIRTIENDO... INFORME 3 C:\Documents\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/15195_CRS_21-05-2025_VEIZAGA ENRIQUEZ GUSTAVO_507.docx
C:\Documents\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/15195_CRS_21-05-2025_VEIZAGA ENRIQUEZ GUSTAVO_507.docx
C:\Documents\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/pdf/15195_CRS_21-05-2025_VEIZAGA ENRIQUEZ GUSTAVO_507.pdf


...

CONVIRTIENDO... INFORME 9 C:\Documents\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/743_CRS_31-05-2025_ARTEAGA CHAVEZ ROBERT EDGAR._493.docx
C:\Documents\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/743_CRS_31-05-2025_ARTEAGA CHAVEZ ROBERT EDGAR._493.docx
C:\Documents\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/pdf/743_CRS_31-05-2025_ARTEAGA CHAVEZ ROBERT EDGAR._493.pdf


  0%|          | 0/1 [00:00<?, ?it/s]

CONVIRTIENDO... INFORME 10 C:\Documents\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/8894_CRS_28-05-2025_MORA MIRANDA MARCELO_551.docx
C:\Documents\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/8894_CRS_28-05-2025_MORA MIRANDA MARCELO_551.docx
C:\Documents\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/pdf/8894_CRS_28-05-2025_MORA MIRANDA MARCELO_551.pdf


  0%|          | 0/1 [00:00<?, ?it/s]

END


# Leer archivos

In [5]:
# CONEXION A GOOGLE DRIVE
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
gauth = GoogleAuth()
gauth.LocalWebserverAuth() # client_secrets.json need to be in the same directory as the script
drive = GoogleDrive(gauth)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=1096764895697-57btfmtn7rdejufct5qq4j9mi61j0unl.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [10]:
contenido = os.listdir(ruta_completa + r'/pdf')
dic_files = {}
for file in contenido:
    split = file.split('_')
    dic_files[int(split[0])] = file
print(len(dic_files), 'informes')
dic_files

6 informes


{180: '180_CRS_16-05-2025_AMURRIO ALBARRACIN ELISEO_481.pdf',
 41819: '41819_CRS_14-05-2025_AGROPECUARIA GUSTAVITO SRL_525.pdf',
 41944: '41944_CRS_14-05-2025_CONDORI QUEVEDO RONALD_546.pdf',
 4914: '4914_CRS_15-05-2025_GONZALEZ LIMPIAS LUIS ALBERTO_490.pdf',
 7506: '7506_CRS_14-05-2025_LOHENER ROSENDA SHEILA PAZ DE_578.pdf',
 7754: '7754_CRS_15-05-2025_LEON MONDAQUE ROLY_554.pdf'}

# CARGAR LOS PDFS A GOOGLE DRIVE

In [11]:
# CARGAR LOS PDFS A GOOGLE DRIVE
dic_urls = {}
# View all folders and file in your Google Drive
fileList = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file in fileList:
  #print('Title: %s, ID: %s' % (file['title'], file['id']))
  # Get the folder ID that you want
  if(file['title'] == "2DA_INSPECCION"):
      fileID = file['id']

for file in dic_files:
    file1 = drive.CreateFile({"mimeType": "application/pdf", "parents": [{"kind": "drive#fileLink", "id": fileID}]})
    file1['title'] = dic_files[file]
    file1.SetContentFile(ruta_completa + r'/pdf//' + dic_files[file])
    file1.Upload() # Upload the file.
    print('Created file %s with mimeType %s' % (file1['title'], file1['mimeType']))
    permission = file1.InsertPermission({
                        'type': 'anyone',
                        'value': 'anyone',
                        'role': 'reader'})
    dic_urls[file] = file1['alternateLink']
dic_urls
print('end')

Created file 180_CRS_16-05-2025_AMURRIO ALBARRACIN ELISEO_481.pdf with mimeType application/pdf
Created file 41819_CRS_14-05-2025_AGROPECUARIA GUSTAVITO SRL_525.pdf with mimeType application/pdf
Created file 41944_CRS_14-05-2025_CONDORI QUEVEDO RONALD_546.pdf with mimeType application/pdf
Created file 4914_CRS_15-05-2025_GONZALEZ LIMPIAS LUIS ALBERTO_490.pdf with mimeType application/pdf
Created file 7506_CRS_14-05-2025_LOHENER ROSENDA SHEILA PAZ DE_578.pdf with mimeType application/pdf
Created file 7754_CRS_15-05-2025_LEON MONDAQUE ROLY_554.pdf with mimeType application/pdf
end


In [12]:
print(len(dic_urls), 'URLs')
dic_urls

6 URLs


{180: 'https://drive.google.com/file/d/1us3hVOaEuupKfR_MGjLhmlxUy7fGOqo4/view?usp=drivesdk',
 41819: 'https://drive.google.com/file/d/1kvJDo79dIz30ezXVD0z1Xyoc3gQik96T/view?usp=drivesdk',
 41944: 'https://drive.google.com/file/d/16DlPA0z1yJlbDXOaBSyO8VfQc8RobFD7/view?usp=drivesdk',
 4914: 'https://drive.google.com/file/d/13_Vxk7t2OwZk9fvMVuo2NPcJ6rOSP718/view?usp=drivesdk',
 7506: 'https://drive.google.com/file/d/1plo0O_hO21j2p6ixzmCOif6cE8PxtS38/view?usp=drivesdk',
 7754: 'https://drive.google.com/file/d/1YgvFylNS-XMLlWeDNfBrsuzooIOX-PMf/view?usp=drivesdk'}

# conexion a amigocloud

In [13]:
import sys
sys.path.append('../../../_amigocloud')

from amigocloud import AmigoCloud

In [14]:
# leer token
acceso = open(ruta_completa + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [15]:
for key in dic_urls:
    query = {'query': 'update dataset_307560 set link_doc_2da = \'{link}\' where cast(SPLIT_PART(canhero, \' / \', 1) as integer) = \'{cod}\''.format(link=dic_urls[key], cod=key)}
    res = amigocloud.post('https://app.amigocloud.com/api/v1/projects/32672/sql', query)
    print(key)
print('end')

180
41819
41944
4914
7506
7754
end
